# This notebook attempts to extract all possible accounts from three *annual* financial reports: the balance sheet, income statement and cash flow statement.

### Why this work?
The output of this work will be used for a table mapping a Vietstock account to a hard-coded FAD account, which will be stored in our database.

### How is a Vietstock account defined?
We define a Vietstock account as a unique combination of its ID (denoted by the field `ReportNormID`), its English name (denoted by the field `NameEn`), and its Vietnamese name (denoted by the field `Name`).

In [1]:
import pandas as pd
import json
import glob
import re

### Here we define a function to simplify account names

In [2]:
def simplify_text(s):
    s1 = s.split(".")[-1]
    s1 = re.sub(r"\W+", " ", s1)
    s1 = " ".join(s1.split())
    s1 = s1.lower()
    return s1

### Define all necessary dictionaries

In [3]:
report_types = {"CDKT":{},
                "KQKD":{},
                "LC":{}}
lookup_dict_all = {"CDKT":{},
                    "KQKD":{},
                    "LC": {}}

### Process each report type sequentially
Before beginning the code, it's helpful to see what a financeInfo report file looks like:

```
[
    [
        {'year': 0000},
        {'year': 0001},
        ...
    ],
    {    
        'CashFlow Indirect': [
            {
                'reportnormID': 1234
            },
            {
                'reportnormID': 1234
            }
        ],
        'CashFlow Indirect': [
            {
                'reportnormID': 5678
            },
            {
                'reportnormID': 9000
            }
        ]
    },
    ...
]
```

In [4]:
for report_type, report_all_accounts in report_types.items():    
    for file in glob.glob(f'functions/schemaData/financeInfo/*{report_type}_Annual*.json'):
        with open(file, 'r') as jsonfile:
            try:
                j = json.load(jsonfile)
                for report_fullname, report_content in j[1].items():
                    if report_fullname not in lookup_dict_all[report_type].keys():
                        lookup_dict_all[report_type][report_fullname] = {}
                    for content in report_content:
                        acc_id = content['ReportNormID']
                        if acc_id not in lookup_dict_all[report_type][report_fullname].keys():
                            lookup_dict_all[report_type][report_fullname][acc_id] = content
            except Exception as e:
                print(f'EXCEPTION: {e}')
    
    for report_fullname, report_content in lookup_dict_all[report_type].items():
        if report_fullname not in report_all_accounts.keys():
            report_all_accounts[report_fullname] = []
        for acc_id, content in report_content.items():
            acc_n = simplify_text(content['NameEn'])
            acc_vi_n = simplify_text(content['Name'])
            parent_n = simplify_text(lookup_dict_all[report_type][report_fullname][content['ParentReportNormID']]['NameEn'])
            parent_vi_n = simplify_text(lookup_dict_all[report_type][report_fullname][content['ParentReportNormID']]['Name'])
            entry = (acc_n, parent_n, acc_vi_n, parent_vi_n)
            if entry not in report_all_accounts[report_fullname]:
                report_all_accounts[report_fullname].append(entry)
    
        with open(f'functions/schema/{report_type}_{report_fullname}_all_accounts.json', 'w') as writefile:
            json.dump(report_all_accounts, writefile, ensure_ascii=False, indent=4)
            
        d = pd.DataFrame(report_all_accounts[report_fullname])
        d.to_excel(f'functions/schema/{report_type}_{report_fullname}_all_accounts.xlsx')

In [ ]:
### Quick view of our results

In [1]:
print(report_types['CDKT'])
print(report_types['KQKD'])

NameError: name 'report_types' is not defined

In [6]:
report_types_expressdetails = {"CDKT":{},
                                "KQKD":{},
                                "LC":{}}
lookup_dict_all_expressdetails = {"CDKT":{},
                                    "KQKD":{},
                                    "LC": {}}

with open('functions/schema/bizType_ind_list.json', 'r') as jsonfile:
    biztypes_inds = sorted(json.load(jsonfile))
    jsonfile.close()

for report_type, report_all_accounts in report_types_expressdetails.items():      
    for biztype_ind in biztypes_inds:
        biztype, ind = biztype_ind.split(";")[0], biztype_ind.split(";")[1]
        if biztype_ind not in lookup_dict_all_expressdetails[report_type].keys():
            lookup_dict_all_expressdetails[report_type][biztype_ind] = {}
        if biztype_ind not in report_types_expressdetails[report_type].keys():
            report_types_expressdetails[report_type][biztype_ind] = {}
        for file in glob.glob(f'functions/schemaData/financeInfo/{biztype}_{ind}*{report_type}_Annual*.json'):
            with open(file, 'r') as jsonfile:
                try:
                    j = json.load(jsonfile)
                    for report_fullname, report_content in j[1].items():
                        if report_fullname not in lookup_dict_all_expressdetails[report_type][biztype_ind].keys():
                            lookup_dict_all_expressdetails[report_type][biztype_ind][report_fullname] = {}
                        for content in report_content:
                            acc_id = content['ReportNormID']
                            if acc_id not in lookup_dict_all_expressdetails[report_type][biztype_ind][report_fullname].keys():
                                lookup_dict_all_expressdetails[report_type][biztype_ind][report_fullname][acc_id] = content
                            
                            ### Test if any of the three elements of the entry are ""
#                             acc_n = simplify_text(content['NameEn'])
#                             acc_vi_n = simplify_text(content['Name'])
#                             acc_parent_id = content['ParentReportNormID']
#                             if (acc_n == "" and acc_vi_n == "") or (acc_n == "" and acc_parent_id == "") or (acc_vi_n == "" and acc_parent_id == ""):
#                                 print(file)
#                                 print(f'{acc_n};{acc_vi_n};{acc_parent_id}')
                                
                except Exception as e:
                    print(f'EXCEPTION: {e}')
                    
        for report_fullname, report_content in lookup_dict_all_expressdetails[report_type][biztype_ind].items():
            if report_fullname not in report_all_accounts[biztype_ind].keys():
                report_all_accounts[biztype_ind][report_fullname] = {}
            for acc_id, content in report_content.items():
                acc_n = simplify_text(content['NameEn'])
                acc_vi_n = simplify_text(content['Name'])
                parent_n = simplify_text(lookup_dict_all_expressdetails[report_type][biztype_ind][report_fullname][content['ParentReportNormID']]['NameEn'])
                parent_vi_n = simplify_text(lookup_dict_all_expressdetails[report_type][biztype_ind][report_fullname][content['ParentReportNormID']]['Name'])
                
                entry = f'{acc_n};{parent_n};{acc_vi_n};{parent_vi_n}'
                if entry not in report_all_accounts[biztype_ind][report_fullname].keys():
                    report_all_accounts[biztype_ind][report_fullname][entry] = [acc_id]
                else:
                    report_all_accounts[biztype_ind][report_fullname][entry].append(acc_id)
                    
            for entry, ids in report_all_accounts[biztype_ind][report_fullname].items():
                if len(ids) > 1:
                    print("=== WARNING: one entry has multiple account IDs ===")
                    print(f'ENTRY "{entry}", BIZ TYPE;INDUSTRY "{biztype_ind}", REPORT "{report_type}-{report_fullname}" \n')

            with open(f'functions/schema/{biztype_ind}_{report_type}_{report_fullname}_all_accounts.json', 'w') as writefile:
                json.dump(report_all_accounts[biztype_ind], writefile, ensure_ascii=False, indent=4)

=== WARNING: one entry has multiple account IDs ===
ENTRY ";operating expenses;;cộng chi phí hoạt động 21 33", BIZ TYPE;INDUSTRY "Security Company;Finance and Insurance", REPORT "KQKD-Income Statement" 

=== WARNING: one entry has multiple account IDs ===
ENTRY ";add non cash expenses;;tăng các chi phí phi tiền tệ", BIZ TYPE;INDUSTRY "Security Company;Finance and Insurance", REPORT "LC-CashFlow Indirect" 

=== WARNING: one entry has multiple account IDs ===
ENTRY "cash at bank for securities company activities;add non cash expenses;tiền gửi ngân hàng cho hoạt động ctck;tăng các chi phí phi tiền tệ", BIZ TYPE;INDUSTRY "Security Company;Finance and Insurance", REPORT "LC-CashFlow Indirect" 

=== WARNING: one entry has multiple account IDs ===
ENTRY "cash equivalents;add non cash expenses;các khoản tương đương tiền;tăng các chi phí phi tiền tệ", BIZ TYPE;INDUSTRY "Security Company;Finance and Insurance", REPORT "LC-CashFlow Indirect" 

=== WARNING: one entry has multiple account IDs ===
E

## Explanation of above warnings:

`";operating expenses;;cộng chi phí hoạt động 21 33"`: empty account En name and empty account Vi name - this is a dummy account used as a padding/empty space bar for the viewing of that financial document

**Solution: ignore this case when processing tickers to database**


`"cash at bank for securities company activities;add non cash expenses;tiền gửi ngân hàng cho hoạt động ctck;tăng các chi phí phi tiền tệ"`: all accounts are not empty - the same account with the same nature has different IDs

**Solution: do not ignore this case**


`";;;phần lưu chuyển tiền tệ hoạt động môi giới ủy thác của khách hàng"`: empty account En name, empty account Vi name, empty parent account En name - this is a dummy account used as a padding/empty space bar for the viewing of that financial document

**Solution: ignore this case when processing tickers to database**


`";;tiền gửi của nhà đầu tư về giao dịch chứng khoán theo phương thức ctck quản lý;phần lưu chuyển tiền tệ hoạt động môi giới ủy thác của khách hàng"`: empty account En name and empty account parent En name - the same account with the same nature has different IDs

**Solution: do not ignore this case**


`"cash equivalents;;các khoản tương đương tiền;phần lưu chuyển tiền tệ hoạt động môi giới ủy thác của khách hàng"`: empty parent account En name

**Solution: do not ignore this case**


### In general, it seems that we can ignore the entries which have empty components at indices `[0]` and `[2]`, which stores the values of `acc_n` and `acc_vi_n`, respectively.